In [14]:
%pylab inline 
import pandas as pd
import glob
from collections import defaultdict
from scipy import stats, signal
#from statsmodels.stats.contingency_tables import Table2x2
import statsmodels.api as sm
import matplotlib.pyplot as plt
import musicbrainzngs as mb
from tqdm import tqdm
import re
import unicodedata


from tqdm.notebook import tqdm

rng = np.random.RandomState(42)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


### Extracting Billboard Artist Gender

In [2]:
# !pip install musicbrainzngs

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Read the billboard csv dataset:
billboard_artist_songs = pd.read_csv('billboard_songs_1960_2023.csv')

In [5]:
billboard_artist_songs.shape

(6400, 4)

In [6]:
billboard_feats = billboard_artist_songs[billboard_artist_songs.Artist.str.contains('featuring')]

In [7]:
billboard_artist_songs[billboard_artist_songs.Artist.str.contains('&')]

,Year,Position,Song Title,Artist
26,1960,27,Baby (You've Got What It Takes),Dinah Washington & Brook Benton
48,1960,49,Finger Poppin' Time,Hank Ballard & The Midnighters
52,1960,53,Theme from The Apartment,Ferrante & Teicher
61,1960,62,Image of a Girl,The Safaris & The Phantom's Band
74,1960,75,A Rockin' Good Way (to Mess Around and Fall in...,Dinah Washington & Brook Benton
...,...,...,...,...
5491,2014,92,White Walls,Macklemore & Ryan Lewis featuring ScHoolboy Q ...
5583,2015,84,Downtown,"Macklemore & Ryan Lewis featuring Eric Nally, ..."
5760,2017,61,Rolex,Ayo & Teo
5762,2017,63,Juju on That Beat (TZ Anthem),Zay Hilfigerrr & Zayion McCall


### MusicBrainz API: Get Gender and Genres for Billboard Artists


In [17]:
mb.set_useragent(app='Artist-to-genre-inator',version='1.0',contact='write_email@email.com')

def normalize_string(input_str):
    # Normalize Unicode characters to their base form (NFD) and remove non-ASCII characters
    normalized_str = unicodedata.normalize('NFD', input_str).encode('ascii', 'ignore').decode('utf-8')
    # Convert to lowercase and remove special characters for better matching
    return re.sub(r'\W+', '', normalized_str).lower()

def find_matching_artist(artist_name, artist_list):
    for artist in artist_list:
        if normalize_string(artist_name) == normalize_string(artist['name']):
            return artist
    return None

def get_artist_info(artist_name):
    result = mb.search_artists(artist=artist_name)
    artist_list = result['artist-list']
    matching_artist = find_matching_artist(artist_name, artist_list)
    artist_info = {'name': artist_name, 'genres': 'Unknown', 'gender': 'Unknown', 'type': 'Unknown'}

    if matching_artist:
        artist_info['type'] = matching_artist.get('type', 'Unknown')
        tags = matching_artist.get('tag-list', [])
        artist_info['genres'] = [tag['name'] for tag in tags if int(tag.get('count', '0')) > 1]
        artist_info['gender'] = matching_artist.get('gender', 'Unknown')

    return artist_info

def get_artist_and_collaboration_info(artist_string):
    result = mb.search_artists(artist=artist_string)
    artist_list = result['artist-list']
    matching_artist = find_matching_artist(artist_string, artist_list)
    keywords = [" featuring ", "featuring", ",", " and ", " ft. ", " ft ", " with ", " & "]
    # Split the string by each keyword
    for keyword in keywords:
        if (keyword in artist_string) & (not matching_artist):
            artist_string = artist_string.replace(keyword, ",")

    # Split by comma and then strip whitespace from each artist's name
    if not matching_artist:
        artists = [artist.strip() for artist in artist_string.split(",") if artist.strip()]
        artist_infos = []
        for artist_name in artists:
            artist_info = get_artist_info(artist_name)
            artist_infos.append(artist_info)
    else:
        artist_infos = []
        artist_infos.append(get_artist_info(artist_string))
        
    return artist_infos

In [18]:
get_artist_and_collaboration_info('NSYNC')

[{'name': 'NSYNC',
  'genres': ['pop', 'dance-pop', 'boy band', 'teen pop'],
  'gender': 'Unknown',
  'type': 'Group'}]

In [8]:
billboard_artist_songs

,Year,Position,Song Title,Artist
0,1960,1,Theme from A Summer Place,Percy Faith
1,1960,2,He'll Have to Go,Jim Reeves
2,1960,3,Cathy's Clown,The Everly Brothers
3,1960,4,Running Bear,Johnny Preston
4,1960,5,Teen Angel,Mark Dinning
...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira
6396,2023,97,Meltdown,Travis Scott featuring Drake
6397,2023,98,Put It on da Floor Again,Latto featuring Cardi B
6398,2023,99,Bloody Mary,Lady Gaga


In [ ]:
def analyze_artist(artist_info):
    if len(artist_info) > 1:
        # Collaboration case
        artist_type = 'Collab'
        genders = {info['gender'] for info in artist_info}
        if 'male' in genders and 'female' in genders:
            gender = 'Mixed'
        elif 'male' in genders:
            gender = 'Male'
        elif 'female' in genders:
            gender = 'Female'
        else:
            gender = 'Unknown'
    else:
        # Individual artist or group
        artist_type = artist_info[0]['type']
        gender = artist_info[0]['gender']

    genres = set()
    for info in artist_info:
        if info['genres'] != 'Unknown':
            genres.update(info['genres'])

    return artist_type, gender, list(genres)

# Iterate over each row and apply the function
for index, row in tqdm(df.iterrows()):
    artist_info = get_artist_and_collaboration_info(row['Artist'])
    artist_type, artist_gender, artist_genres = analyze_artist(artist_info)

    df.at[index, 'Artist Type'] = artist_type
    df.at[index, 'Artist Gender'] = artist_gender
    df.at[index, 'Genres'] = ', '.join(artist_genres)

# Save the updated DataFrame to a new CSV file
# df.to_csv('billboard_songs_1960_2023_with_artist_gender_and_genre.csv', index=False)

In [22]:
billboard_df = pd.read_csv('billboard_songs_1960_2023_with_artist_gender_and_genre.csv')

In [23]:
billboard_df

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,song_title,artist_name
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,theme from a summer place,percy faith
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,country,he ll have to go,jim reeves
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"rockabilly, rock & roll",cathy s clown,the everly brothers
3,1960,4,Running Bear,Johnny Preston,Person,Male,NaN,running bear,johnny preston
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,teen angel,mark dinning
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"pop, pop rock, latin pop",bzrp music sessions vol 53,bizarrap and shakira
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"english, r&b, contemporary r&b, alternative r&...",meltdown,travis scott feat drake
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"alternative pop, awful, hip hop",put it on da floor again,latto feat cardi b
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"dance, synth-pop, disco, pop, pop rock, electr...",bloody mary,lady gaga


In [52]:
billboard_df.isna().sum()

Year                0
Position            0
Song Title          0
Artist              0
Artist Type         0
Artist Gender       0
Genres           1917
dtype: int64

In [56]:
billboard_df[billboard_df['Artist']=='Hamilton, Joe Frank & Reynolds']

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
1141,1971,42,Don't Pull Your Love,"Hamilton, Joe Frank & Reynolds",Group,Unknown,NaN
1544,1975,45,Fallin' in Love,"Hamilton, Joe Frank & Reynolds",Group,Unknown,NaN


In [57]:
billboard_df[(billboard_df['Artist Type']=='Person') & (billboard_df['Artist Gender']=='Unknown')]

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
80,1960,81,Tracy's Theme,Spencer Ross,Person,Unknown,NaN
278,1962,79,Dear One,Larry Finnegan,Person,Unknown,NaN
280,1962,81,Old Rivers,Walter Brennan,Person,Unknown,NaN
493,1964,94,Steal Away,Jimmy Hughes,Person,Unknown,NaN
1133,1971,34,Chick-A-Boom (Don't Ya Jes' Love It),Daddy Dewdrop,Person,Unknown,NaN
1285,1972,86,Don't Say You Don't Remember,Beverly Bremers,Person,Unknown,NaN
4183,2001,84,I Wanna Be Bad,Willa Ford,Person,Unknown,NaN
4715,2007,16,This Is Why I'm Hot,Mims,Person,Unknown,hip hop


In [58]:
billboard_df[(billboard_df['Artist Type']=='Collab') & (billboard_df['Artist Gender']=='Unknown')]

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
52,1960,53,Theme from The Apartment,Ferrante & Teicher,Collab,Unknown,NaN
61,1960,62,Image of a Girl,The Safaris & The Phantom's Band,Collab,Unknown,NaN
116,1961,17,Exodus,Ferrante & Teicher,Collab,Unknown,NaN
126,1961,27,The Mountain's High,Dick and Dee Dee,Collab,Unknown,NaN
140,1961,41,Daddy's Home,Shep and the Limelites,Collab,Unknown,NaN
...,...,...,...,...,...,...,...
5704,2017,5,Something Just Like This,The Chainsmokers and Coldplay,Collab,Unknown,"uk, pop, alternative rock, british, pop rock, ..."
5705,2017,6,Bad and Boujee,Migos featuring Lil Uzi Vert,Collab,Unknown,"east coast hip hop, trap, hip hop"
5790,2017,91,No Promises,Cheat Codes featuring Demi Lovato,Collab,Unknown,"pop, pop rock, dance-pop, teen pop, electropop"
5865,2018,66,Lights Down Low,Max featuring Gnash,Collab,Unknown,NaN


In [59]:
billboard_df[(billboard_df['Artist Type']=='Group') & (billboard_df['Artist Gender']=='Unknown')]

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Unknown,"rockabilly, rock & roll"
12,1960,13,Greenfields,The Brothers Four,Group,Unknown,NaN
15,1960,16,Alley Oop,The Hollywood Argyles,Group,Unknown,NaN
24,1960,25,"Walk, Don't Run",The Ventures,Group,Unknown,NaN
25,1960,26,Save the Last Dance for Me,The Drifters,Group,Unknown,soul
...,...,...,...,...,...,...,...
6236,2022,37,I Ain't Worried,OneRepublic,Group,Unknown,pop
6281,2022,82,Take My Name,Parmalee,Group,Unknown,NaN
6343,2023,44,Cupid,Fifty Fifty,Group,Unknown,"girl group, k-pop"
6383,2023,84,Last Christmas,Wham!,Group,Unknown,"pop, dance-pop"


In [75]:
billboard_df['artist_name'] = billboard_df['Artist']

In [76]:
billboard_df['artist_name'] = billboard_df['artist_name'].apply(normalize_string)

In [77]:
billboard_df.head(3)

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,artist_name
0,1960,1,Theme from A Summer Place,Percy Faith,Person,male,NaN,percy faith
1,1960,2,He'll Have to Go,Jim Reeves,Person,male,country,jim reeves
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Unknown,"rockabilly, rock & roll",the everly brothers


In [162]:
wasabi_artists_with_gender = wasabi_song_lyrics.dropna(subset = ['artist_name', 'artist_type', 'artist_gender'])
wasabi_artists_with_gender.drop_duplicates(subset = ['artist_name', 'artist_type', 'artist_gender'], inplace = True)
wasabi_artists_with_gender.reset_index(drop = True, inplace = True)
wasabi_artists_with_gender.shape

(11425, 64)

In [164]:
wasabi_artists_with_gender['artist_name'] = wasabi_artists_with_gender['artist_name'].apply(normalize_string)

In [165]:
wasabi_artists_with_gender = wasabi_artists_with_gender[['artist_name', 'artist_type', 'artist_gender']]

In [166]:
wasabi_artists_with_gender.reset_index(drop = True, inplace = True)
wasabi_artists_with_gender.rename(columns = {'artist_type': 'Artist Type', 'artist_gender': 'Artist Gender'}
                               , inplace = True)
wasabi_artists_with_gender

/tmp/3278552.1.all.q/ipykernel_4809/1315177154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wasabi_artists_with_gender.rename(columns = {'artist_type': 'Artist Type', 'artist_gender': 'Artist Gender'}


,artist_name,Artist Type,Artist Gender
0,aaron neville,Person,Male
1,al di meola,Person,Male
2,al green,Person,Male
3,al kooper,Person,Male
4,al wilson,Person,Male
...,...,...,...
11420,young money,Group,Female
11421,yourcodenameis milo,Group,Male
11422,yves klein blue,Group,Male
11423,zahl,Group,Male


In [167]:
billboard_df

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,artist_name
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,percy faith
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,country,jim reeves
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Unknown,"rockabilly, rock & roll",the everly brothers
3,1960,4,Running Bear,Johnny Preston,Person,Male,NaN,johnny preston
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,mark dinning
...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"pop, pop rock, latin pop",bizarrap and shakira
6396,2023,97,Meltdown,Travis Scott featuring Drake,Collab,Male,"english, r&b, contemporary r&b, alternative r&...",travis scott featuring drake
6397,2023,98,Put It on da Floor Again,Latto featuring Cardi B,Collab,Female,"alternative pop, awful, hip hop",latto featuring cardi b
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"dance, synth-pop, disco, pop, pop rock, electr...",lady gaga


In [168]:
wasabi_billboard_groups['Artist Gender'].value_counts()

Artist Gender
Male      497
Mix       117
Female     55
Name: count, dtype: int64

In [169]:
wasabi_billboard_groups.isna().sum()

artist_name      0
Artist Type      0
Artist Gender    0
dtype: int64

In [170]:
billboard_df['Artist Type'].value_counts()

Artist Type
Person       3064
Group        2083
Collab       1119
Unknown       131
Orchestra       2
Other           1
Name: count, dtype: int64

In [171]:
billboard_df['Artist Gender'] = df['Artist Gender'].str.capitalize()

In [172]:
billboard_df['Artist Gender'].value_counts()

Artist Gender
Male          2503
Unknown       2292
Female        1228
Mixed          352
Non-binary      25
Name: count, dtype: int64

In [173]:
billboard_df_merge = billboard_df.merge(wasabi_artists_with_gender, on = ['artist_name', 'Artist Type'], how ='left')

In [174]:
billboard_df_merge

,Year,Position,Song Title,Artist,Artist Type,Artist Gender_x,Genres,artist_name,Artist Gender_y
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,percy faith,NaN
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,country,jim reeves,Male
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Unknown,"rockabilly, rock & roll",the everly brothers,Male
3,1960,4,Running Bear,Johnny Preston,Person,Male,NaN,johnny preston,Male
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,mark dinning,NaN
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"pop, pop rock, latin pop",bizarrap and shakira,NaN
6396,2023,97,Meltdown,Travis Scott featuring Drake,Collab,Male,"english, r&b, contemporary r&b, alternative r&...",travis scott featuring drake,NaN
6397,2023,98,Put It on da Floor Again,Latto featuring Cardi B,Collab,Female,"alternative pop, awful, hip hop",latto featuring cardi b,NaN
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"dance, synth-pop, disco, pop, pop rock, electr...",lady gaga,Female


In [175]:
billboard_df_merge.loc[billboard_df_merge['Artist Gender_x'] == 'Unknown', 'Artist Gender_x'] = billboard_df_merge['Artist Gender_y']
billboard_df_merge = billboard_df_merge.drop(columns=['Artist Gender_y', 'artist_name'])
billboard_df_merge.rename(columns = {'Artist Gender_x':'Artist Gender'}, inplace = True)
billboard_df_merge

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,country
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"rockabilly, rock & roll"
3,1960,4,Running Bear,Johnny Preston,Person,Male,NaN
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN
...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"pop, pop rock, latin pop"
6396,2023,97,Meltdown,Travis Scott featuring Drake,Collab,Male,"english, r&b, contemporary r&b, alternative r&..."
6397,2023,98,Put It on da Floor Again,Latto featuring Cardi B,Collab,Female,"alternative pop, awful, hip hop"
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"dance, synth-pop, disco, pop, pop rock, electr..."


In [176]:
billboard_df_merge.isna().sum()

Year                0
Position            0
Song Title          0
Artist              0
Artist Type         0
Artist Gender    1352
Genres           1917
dtype: int64

In [177]:
billboard_df_merge['Artist Gender'].value_counts()

Artist Gender
Male          3225
Female        1296
Mixed          352
Mix            150
Non-binary      25
Name: count, dtype: int64

In [178]:
billboard_df_merge['Artist Gender'] = billboard_df_merge['Artist Gender'].replace('Mix', 'Mixed')

In [179]:
billboard_df_merge['Artist Gender'].value_counts()

Artist Gender
Male          3225
Female        1296
Mixed          502
Non-binary      25
Name: count, dtype: int64

In [191]:
billboard_df_merge['Artist'] = billboard_df_merge['Artist'].str.replace('featuring', 'feat.', regex=False)

In [198]:
billboard_df_merge

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,country
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"rockabilly, rock & roll"
3,1960,4,Running Bear,Johnny Preston,Person,Male,NaN
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN
...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"pop, pop rock, latin pop"
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"english, r&b, contemporary r&b, alternative r&..."
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"alternative pop, awful, hip hop"
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"dance, synth-pop, disco, pop, pop rock, electr..."


In [194]:
billboard_df_merge.to_csv('billboard_songs_1960_2023_with_artist_gender_and_genre.csv', index = None)

## Get the lyrics of the songs via Genius API:

In [73]:
# !pip install lyricsgenius

In [148]:
import lyricsgenius

In [149]:
billboard_artist_lyrics = pd.read_csv('billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv')

In [154]:
import time
import lyricsgenius
from requests.exceptions import Timeout

def get_lyrics(song_title, artist_name, genius_api):
    genius = lyricsgenius.Genius(access_token)
    retries = 0
    while retries < 3:
        try:
            # Search for the song and artist
            song = genius.search_song(song_title, artist_name)
            
            # Check if the song is found and titles match
            if song and normalize_string(song.title) == normalize_string(song_title):
                return song.lyrics
            else:
                return "Song not found."
        except Timeout:
            retries += 1
            time.sleep(1)  # Delay before retrying
            continue
        except Exception as e:
            return f"An error occurred: {e}"

    return "Request timed out after 3 retries."

In [11]:
access_token = '' # add your own access token
genius_api = lyricsgenius.Genius(access_token)


# Iterate over each row in the dataset
for index, row in tqdm(billboard_artist_lyrics.iterrows()):
    if pd.isna(billboard_artist_lyrics.loc[index, 'lyrics']):
        lyrics = get_lyrics(row['Song Title'], row['Artist'], genius_api)
        billboard_artist_lyrics.at[index, 'lyrics'] = lyrics

# Save the updated dataset
billboard_artist_lyrics.to_csv('billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv', index=False)  

Specified song does not contain lyrics. Rejecting.
Searching for "Hi-Heel Sneakers" by Tommy Tucker...
Done.
Searching for "How Do You Do It?" by Gerry and the Pacemakers...
Done.
Searching for "Walk, Don't Run '64" by The Ventures...
Specified song does not contain lyrics. Rejecting.
Searching for "Do You Love Me" by The Dave Clark Five...
Done.
Searching for "Shangri-La" by Robert Maxwell...
Done.
Searching for "Haunted House" by Jumpin' Gene Simmons...
Done.
Searching for "Steal Away" by Jimmy Hughes...
Done.
Searching for "Bad to Me" by Billy J. Kramer & The Dakotas...
Done.
Searching for "Wooly Bully" by Sam the Sham and the Pharaohs...
Done.
Searching for "I Can't Help Myself (Sugar Pie Honey Bunch)" by Four Tops...
Done.
Searching for "You Were on My Mind" by We Five...
Done.
Searching for "You've Lost That Lovin' Feelin'" by The Righteous Brothers...
Done.
Searching for "Can't You Hear My Heartbeat" by Herman's Hermits...
Done.
Searching for "My Girl" by The Temptations...
Done

Done.
Searching for "Expressway to Your Heart" by Soul Survivors...
Done.
Searching for "Soul Man" by Sam & Dave...
Done.
Searching for "Never My Love" by The Association...
Done.
Searching for "Apples, Peaches, Pumpkin Pie" by Jay & the Techniques...
Done.
Searching for "Come On Down to My Boat" by Every Mother's Son...
Done.
Searching for "Love Is Here and Now You're Gone" by The Supremes...
Done.
Searching for "For What It's Worth" by Buffalo Springfield...
Done.
Searching for "The Happening" by The Supremes...
Done.
Searching for "Your Precious Love" by Marvin Gaye & Tammi Terrell...
Done.
Searching for "Get on Up" by The Esquires...
Done.
Searching for "Jimmy Mack" by Martha and the Vandellas...
Done.
Searching for "I Got Rhythm" by The Happenings...
Done.
Searching for "A Whiter Shade of Pale" by Procol Harum...
Searching for "A Whiter Shade of Pale" by Procol Harum...
Searching for "A Whiter Shade of Pale" by Procol Harum...
Done.
Searching for "Then You Can Tell Me Goodbye" by 

Done.
Searching for "The Love You Save" by The Jackson 5...
Searching for "The Love You Save" by The Jackson 5...
Done.
Searching for "Candida" by Dawn...
Done.
Searching for "Thank You (Falettinme Be Mice Elf Agin)" by Sly & the Family Stone...
Done.
Searching for "Spill the Wine" by Eric Burdon & War...
Done.
Searching for "Lay Down (Candles in the Rain)" by Melanie...
Done.
Searching for "Ball of Confusion (That's What the World Is Today)" by The Temptations...
Done.
Searching for "Love on a Two-Way Street" by The Moments...
Searching for "Love on a Two-Way Street" by The Moments...
Done.
Searching for "All Right Now" by Free...
Searching for "All Right Now" by Free...
Done.
Searching for "I Want You Back" by The Jackson 5...
Searching for "I Want You Back" by The Jackson 5...
Done.
Searching for "Julie, Do Ya Love Me" by Bobby Sherman...
Done.
Searching for "Green-Eyed Lady" by Sugarloaf...
Done.
Searching for "Venus" by Shocking Blue...
Done.
Searching for "Instant Karma!" by John

Done.
Searching for "Hurting Each Other" by The Carpenters...
Done.
Searching for "Joy" by Apollo 100...
Specified song does not contain lyrics. Rejecting.
Searching for "Family Affair" by Sly & the Family Stone...
Done.
Searching for "Troglodyte (Cave Man)" by The Jimmy Castor Bunch...
Done.
Searching for "The Witch Queen of New Orleans" by Redbone...
Done.
Searching for "Ain't Understanding Mellow" by Jerry Butler & Brenda Lee Eager...
Done.
Searching for "Don't Say You Don't Remember" by Beverly Bremers...
Done.
Searching for "Day by Day" by Original Cast of Godspell...
Done.
Searching for "I'd Like to Teach the World to Sing (In Perfect Harmony)" by The New Seekers...
Done.
Searching for "Baby Let Me Take You (In My Arms)" by The Detroit Emeralds...
Done.
Searching for "I'd Like to Teach the World to Sing (In Perfect Harmony)" by The Hillside Singers...
Done.
Searching for "Walkin' in the Rain with the One I Love" by Love Unlimited...
Done.
Searching for "Tie a Yellow Ribbon Round 

Done.
Searching for "Get Dancin'" by Disco-Tex and the Sex-O-Lettes...
Done.
Searching for "Don't Go Breaking My Heart" by Elton John & Kiki Dee...
Searching for "Don't Go Breaking My Heart" by Elton John & Kiki Dee...
Done.
Searching for "December, 1963 (Oh, What a Night)" by The Four Seasons...
Searching for "December, 1963 (Oh, What a Night)" by The Four Seasons...
Searching for "December, 1963 (Oh, What a Night)" by The Four Seasons...
Done.
Searching for "Kiss and Say Goodbye" by The Manhattans...
Done.
Searching for "A Fifth of Beethoven" by Walter Murphy & The Big Apple Band...
Specified song does not contain lyrics. Rejecting.
Searching for "Sara Smile" by Hall & Oates...
Done.
Searching for "Fly, Robin, Fly" by Silver Convention...
Done.
Searching for "Get Closer" by Seals and Crofts...
Done.
Searching for "Boogie Fever" by The Sylvers...
Done.
Searching for "You Sexy Thing" by Hot Chocolate...
Searching for "You Sexy Thing" by Hot Chocolate...
Done.
Searching for "Love Hurts"

...

Done.
Searching for "Treat You Better" by Shawn Mendes...
Done.
Searching for "I Get the Bag" by Gucci Mane feat. Migos...
Done.
Searching for "Small Town Boy" by Dustin Lynch...
Done.
Searching for "Everyday We Lit" by YFN Lucci feat. PnB Rock...
Done.
Searching for "Havana" by Camila Cabello feat. Young Thug...
Done.
Searching for "What Lovers Do" by Maroon 5 feat. SZA...
Done.
Searching for "Do Re Mi" by Blackbear...
Done.
Searching for "Look at Me!" by XXXTentacion...
Searching for "Look at Me!" by XXXTentacion...
Done.
Searching for "The Fighter" by Keith Urban feat. Carrie Underwood...
Done.
Searching for "God's Plan" by Drake...
Searching for "God's Plan" by Drake...
Done.
Searching for "Perfect" by Ed Sheeran...
Done.
Searching for "Meant to Be" by Bebe Rexha feat. Florida Georgia Line...
Done.
Searching for "Havana" by Camila Cabello feat. Young Thug...
Done.
Searching for "Rockstar" by Post Malone feat. 21 Savage...
Done.
Searching for "Psycho" by Post Malone feat. Ty Dolla S

Done.
Searching for "Girls Like You" by Maroon 5 feat. Cardi B...
Done.
Searching for "Sweet but Psycho" by Ava Max...
Searching for "Sweet but Psycho" by Ava Max...
Done.
Searching for "Suge" by DaBaby...
Done.
Searching for "Middle Child" by J. Cole...
Done.
Searching for "Drip Too Hard" by Lil Baby and Gunna...
Done.
Searching for "Someone You Loved" by Lewis Capaldi...
Searching for "Someone You Loved" by Lewis Capaldi...
Done.
Searching for "Ransom" by Lil Tecca...
Done.
Searching for "If I Can't Have You" by Shawn Mendes...
Done.
Searching for "Goodbyes" by Post Malone feat. Young Thug...
Done.
Searching for "Zeze" by Kodak Black feat. Travis Scott and Offset...
Done.
Searching for "Better Now" by Post Malone...
Done.
Searching for "Youngblood" by 5 Seconds of Summer...
Done.
Searching for "Money in the Grave" by Drake feat. Rick Ross...
Done.
Searching for "Speechless" by Dan + Shay...
Done.
Searching for "Break Up with Your Girlfriend, I'm Bored" by Ariana Grande...
Done.
Searc

Done.
Searching for "We Paid" by Lil Baby and 42 Dugg...
Done.
Searching for "Yummy" by Justin Bieber...
Done.
Searching for "One Man Band" by Old Dominion...
Done.
Searching for "Got What I Got" by Jason Aldean...
Done.
Searching for "Sunday Best" by Surfaces...
Done.
Searching for "Godzilla" by Eminem feat. Juice Wrld...
Done.
Searching for "Bandit" by Juice Wrld and YoungBoy Never Broke Again...
Done.
Searching for "Party Girl" by StaySolidRocky...
Done.
Searching for "Die from a Broken Heart" by Maddie & Tae...
Done.
Searching for "Popstar" by DJ Khaled feat. Drake...
Done.
Searching for "One of Them Girls" by Lee Brice...
Done.
Searching for "Hard to Forget" by Sam Hunt...
Done.
Searching for "One Margarita" by Luke Bryan...
Done.
Searching for "Panini" by Lil Nas X...
Done.
Searching for "Hot" by Young Thug feat. Gunna...
Done.
Searching for "I Hope You're Happy Now" by Carly Pearce and Lee Brice...
Done.
Searching for "Emotionally Scarred" by Lil Baby...
Done.
Searching for "Sui

Done.
Searching for "Single Saturday Night" by Cole Swindell...
Done.
Searching for "Things a Man Oughta Know" by Lainey Wilson...
Done.
Searching for "Throat Baby (Go Baby)" by BRS Kash...
Done.
Searching for "Tombstone" by Rod Wave...
Done.
Searching for "Drinkin' Beer. Talkin' God. Amen." by Chase Rice feat. Florida Georgia Line...
Done.
Searching for "Todo de Ti" by Rauw Alejandro...
Done.
Searching for "Heat Waves" by Glass Animals...
Done.
Searching for "As It Was" by Harry Styles...
Done.
Searching for "Stay" by The Kid Laroi and Justin Bieber...
Done.
Searching for "Easy on Me" by Adele...
Done.
Searching for "Shivers" by Ed Sheeran...
Done.
Searching for "First Class" by Jack Harlow...
Done.
Searching for "Big Energy" by Latto...
Done.
Searching for "Ghost" by Justin Bieber...
Done.
Searching for "Super Gremlin" by Kodak Black...
Done.
Searching for "Cold Heart (Pnau remix)" by Elton John and Dua Lipa...
Done.
Searching for "Wait for U" by Future feat. Drake and Tems...
Done.


Done.
Searching for "One Thing at a Time" by Morgan Wallen...
Done.
Searching for "Superhero (Heroes & Villains)" by Metro Boomin, Future and Chris Brown...
Done.
Searching for "Bad Habit" by Steve Lacy...
Done.
Searching for "La Bebé" by Yng Lvcas and Peso Pluma...
Done.
Searching for "Golden Hour" by Jvke...
Done.
Searching for "Religiously" by Bailey Zimmerman...
Done.
Searching for "Spin Bout U" by Drake and 21 Savage...
Done.
Searching for "Cupid" by Fifty Fifty...
Done.
Searching for "Search & Rescue" by Drake...
Done.
Searching for "Barbie World" by Nicki Minaj and Ice Spice with Aqua...
Done.
Searching for "Next Thing You Know" by Jordan Davis...
Done.
Searching for "Escapism" by Raye feat. 070 Shake...
Done.
Searching for "Un x100to" by Grupo Frontera and Bad Bunny...
Done.
Searching for "Until I Found You" by Stephen Sanchez...
Done.
Searching for "Shirt" by SZA...
Done.
Searching for "Paint the Town Red" by Doja Cat...
Done.
Searching for "Made You Look" by Meghan Trainor...

### Let's read the dataset:

In [164]:
billboard_artist_lyrics = pd.read_csv('billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv') 

In [165]:
billboard_artist_lyrics

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,3 ContributorsTheme From A Summer Place Lyrics...
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],"On the bank of the river\nStood Running Bear, ..."
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,5 ContributorsTeen Angel Lyrics[Intro]\nTeen A...
...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"['Pop', 'Pop Rock', 'Latin Pop']",Song not found.
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",Song not found.
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",63 ContributorsPut It On Da Floor Again Lyrics...
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",111 ContributorsTranslationsEspañolPortuguêsČe...


### Let's Clean the lyrics:

In [167]:
import re

def clean_lyrics(df, column):

    # Remove bracketed and parenthetical sections, URLs
    df[column] = df[column].str.replace(r"[\(\[].*?[\)\]]|http\S+|www.\S+", "", regex=True)

    # Remove specified words and punctuation
    df[column] = df[column].str.replace(r"verse [1|2|3]|chorus|bridge|outro|instrumental|intro|guitar|solo", "", regex=True)
    df[column] = df[column].str.replace(r"[^\w\d'\s.!?]+", "", regex=True)

    # Remove specific unwanted strings and trailing spaces
    unwanted_strings = ["urlcopyembedcopy", "embedshare", "EmbedShare", "URLCopyEmbedCopy"]
    for string in unwanted_strings:
        df[column] = df[column].str.replace(string, "", regex=False)
    
    df[column] = df[column].str.strip()

    return df


In [168]:
billboard_artist_lyrics['lyrics'] = billboard_artist_lyrics['lyrics'].apply(clean_lyrics)

In [172]:
billboard_artist_lyrics

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...
...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"['Pop', 'Pop Rock', 'Latin Pop']",Song not found
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",Song not found
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",Go Grizz\nAh \nWhat's happenin'? \nBig Latto \...
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",Money\n\n\nLove is just a history that they ma...


### Detect the language of the songs:

In [181]:
# !pip install langdetect

In [9]:
from langdetect import detect

def detect_language_langdetect(text):
    try:
        detected_language = detect(text)
        return detected_language
    except Exception as e:
        print("An error occurred:", e)
        return None

In [189]:
billboard_artist_lyrics['detected_language'] =np.nan
for i in tqdm(range(billboard_artist_lyrics.shape[0])):
    if  pd.isna(billboard_artist_lyrics.loc[i, 'lyrics']):
        billboard_artist_lyrics['detected_language'][i] = np.nan
    else:    
        billboard_artist_lyrics['detected_language'][i]  = detect_language_langdetect(billboard_artist_lyrics['lyrics'][i])

  0%|          | 0/6400 [00:00<?, ?it/s]

/tmp/3291560.1.all.q/ipykernel_12459/3681523245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  billboard_artist_lyrics['detected_language'][i]  = detect_language_langdetect(billboard_artist_lyrics['lyrics'][i])
/tmp/3291560.1.all.q/ipykernel_12459/3681523245.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'en' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  billboard_artist_lyrics['detected_language'][i]  = detect_language_langdetect(billboard_artist_lyrics['lyrics'][i])


In [190]:
billboard_artist_lyrics

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...,en
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...,en
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...,en
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...,en
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...,en
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"['Pop', 'Pop Rock', 'Latin Pop']",NaN,NaN
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",NaN,NaN
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",Go Grizz\nAh \nWhat's happenin'? \nBig Latto \...,en
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",Money\n\n\nLove is just a history that they ma...,en


In [191]:
billboard_artist_lyrics['detected_language'].value_counts()

detected_language
en    5884
es      32
so       4
nl       4
de       3
id       3
tl       3
af       2
cy       2
ko       2
pt       1
da       1
sw       1
sk       1
ja       1
fr       1
ca       1
sv       1
Name: count, dtype: int64

### Now we save the dataset:

In [193]:
# billboard_artist_lyrics.to_csv('billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv', index = None)

In [67]:
billboard_artist_lyrics = pd.read_csv('billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv')

In [68]:
billboard_artist_lyrics.head()

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...,en
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...,en
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...,en
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...,en
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...,en


In [69]:
billboard_artist_lyrics.isna().sum()

Year                    0
Position                0
Song Title              0
Artist                  0
Artist Type             0
Artist Gender        1352
Genres               1232
lyrics                453
detected_language     453
dtype: int64

In [70]:
unknown_gender = billboard_artist_lyrics[billboard_artist_lyrics['Artist Gender'].isna()]

In [71]:
unknown_gender.reset_index(drop = True, inplace = True)

In [72]:
unknown_gender

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,16,Alley Oop,The Hollywood Argyles,Group,NaN,NaN,Oopoop oop oopoop\n\n\n\nThere's a man in the ...,en
1,1960,26,Save the Last Dance for Me,The Drifters,Group,NaN,['Soul'],You can dance\nEvery dance with the guy\nWho g...,en
2,1960,36,The Big Hurt,Toni Fisher,Unknown,NaN,NaN,Now it begins now that you're gone\nNeedles an...,en
3,1960,44,Mule Skinner Blues,The Fendermen,Group,NaN,NaN,Well good morning Captain\nWell good morning t...,en
4,1960,53,Theme from The Apartment,Ferrante & Teicher,Collab,NaN,NaN,1 ContributorTheme From The Apartment LyricsMa...,en
...,...,...,...,...,...,...,...,...,...
1347,2023,44,Cupid,Fifty Fifty,Group,NaN,"['Girl Group', 'K-Pop']",Intro Aran\nLa la la lalala\nLa lalala la lala...,en
1348,2023,73,Bebe Dame,Fuerza Regida and Grupo Frontera,Collab,NaN,NaN,Intro Jesús Paz Payo Solís\nFierro\nÁnimo\nDe...,es
1349,2023,78,Rich Men North of Richmond,Oliver Anthony Music,Unknown,NaN,NaN,I've been sellin' my soul workin' all day\nOve...,en
1350,2023,84,Last Christmas,Wham!,Group,NaN,"['Pop', 'Dance-Pop']",Happy Christmas\nAh ahah\nOohwoah\nOhoh\n\n\nL...,en


### Manual Annotations of Artist Gender (for missing data):

In [73]:
unknown_gender = pd.read_csv('Billboard_song_artist_unk_gender.csv')

In [10]:
artist_gender_annotations = {
    'The Hollywood Argyles': 'Male Group',
    'The Drifters': 'Male Group',
    'Toni Fisher': 'Female',
    'The Fendermen': 'Male Duo',
    'Ferrante & Teicher': 'Male Duo',
    "Bill Black's Combo": 'Male Group',
    'The Four Preps': 'Male Group',
    'The Little Dippers': 'Mixed',  # Adjust based on further research if needed
    "The Safaris & The Phantom's Band": 'Male Group',
    'Skip & Flip': 'Male Duo',
   'Spencer Ross': 'Male',
    'The Crests': 'Male Group',
    'Johnny and the Hurricanes': 'Male Group',
    'The Jive Five': 'Male Group',
    'The String-A-Longs': 'Male Group',
    'The Shirelles': 'Female Group',
    'The Mar-Keys': 'Male Group',
    'The Dovells': 'Male Group',
    'Dick and Dee Dee': 'Mixed Duo',  # Male and Female
    'The Marcels': 'Male Group',
    'Shep and the Limelites': 'Male Group',
    'Kokomo': 'Male',  # Ambiguous without further context
    'The Capris': 'Male Group',
    'The Paris Sisters': 'Female Group',
    'The Lettermen': 'Male Group',
    'The Echoes': 'Male Group',
    'Little Caesar & the Romans': 'Male Group',
    'Rosie and the Originals': 'Mixed Group',  # Female lead
    'The Dave Brubeck Quartet': 'Male Group',
    'The Sensations': 'Mixed Group',  # Typically featured a female lead
    'The Orlons': 'Mixed Group',  # Primarily female leads
    'The Corsairs': 'Male Group',
    'Booker T. & the M.G.\'s': 'Male Group',
    'The Four Seasons': 'Male Group',
    'The Duprees': 'Male Group',
    'Larry Finnegan': 'Male',
    'Walter Brennan': 'Male',
    'Billy Joe and the Checkmates': 'Male Group',
    'The Ikettes': 'Female Group',
    'Peter, Paul and Mary': 'Mixed Group',  # Two males, one female
    'Ronnie & the Hi-Lites': 'Male Group',
    'The Cascades': 'Male Group',
    'The Chiffons': 'Female Group',
    'Paul & Paula': 'Mixed Duo',  # Male and Female
    'Little Stevie Wonder': 'Male',
    'The Angels': 'Female Group',
    'Kyu Sakamoto': 'Male',
    'The Tymes': 'Male Group',
    'The Rebels': 'Male Group',  # Assuming specific group; multiple possibilities
    'Little Peggy March': 'Female',
    'The Chantays': 'Band (Mixed)',
 'Martha and the Vandellas': 'Female Group',
 'Randy & the Rainbows': 'Male Group',
 'The Essex': 'Mixed Group',
 'Ruby & the Romantics': 'Mixed Group',
 'Garnet Mimms & the Enchanters': 'Male Group',
 'The Jaynetts': 'Female Group',
 'Nino Tempo & April Stevens': 'Mixed Duo',
 'The Cookies': 'Female Group',
 'The Village Stompers': 'Band (Mixed)',
 'Sunny & the Sunglows': 'Male Group',
 'The Impressions': 'Male Group',
 'The Dartells': 'Male Group',
 'The Ran-Dells': 'Male Group',
 'Los Indios Tabajaras': 'Male Duo',
 'The New Christy Minstrels': 'Mixed Group',
 'The Supremes': 'Female Group',
 'The Ray Charles Singers': 'Choir (Mixed)',
 'The Dixie Cups': 'Female Group',
 'The Dave Clark Five': 'Male Group',
 'The Newbeats': 'Male Group',
 'The Honeycombs': 'Mixed Group',
 'The Serendipity Singers': 'Mixed Group',
 'The Animals': 'Male Group',
 'Ronny & the Daytonas': 'Male Group',
 'The Rip Chords': 'Male Group',
 'Gerry and the Pacemakers': 'Male Group',
 'Four Tops': 'Male Group',
 'The Bachelors': 'Male Group',
 'The Marketts': 'Band (Mixed)',
 'The Hondells': 'Male Group',
 'The Reflections': 'Male Group',
 'The Temptations': 'Male Group',
 'The Trashmen': 'Male Group',
 'The Tams': 'Male Group',
 'The Jelly Beans': 'Female Group',
 "Jumpin' Gene Simmons": 'Male',
 'Jimmy Hughes': 'Male',
 'We Five': 'Mixed Group',
 'The Righteous Brothers': 'Male Duo',
 "Herman's Hermits": 'Male Group',
 'Gary Lewis & the Playboys': 'Male Group',
 'Ramsey Lewis Trio': 'Male Group',
 'The Seekers': 'Mixed Group',
 'The Byrds': 'Male Group',
 'Sounds Orchestral': 'Orchestra (Mixed)',
 'Wayne Fontana & The Mindbenders': 'Male Group',
 'The Moody Blues': 'Male Group',
 'The Beau Brummels': 'Male Group',
 'The Zombies': 'Male Group',
 "Sonny": "Male",
    "Dino, Desi & Billy": "Male Group",
    "The Ad Libs": "Mixed Group",
    "? and the Mysterians": "Male Group",
    "The Association": "Male Group",
    "SSgt Barry Sadler": "Male",
    "The Young Rascals": "Male Group",
    "The Happenings": "Male Group",
    "The Mindbenders": "Male Group",
    "The Cyrkle": "Male Group",
    "The Capitols": "Male Group",
    "The Outsiders": "Male Group",
    "The Hollies": "Male Group",
    "B. J. Thomas & the Triumphs": "Male Group",
    "Simon & Garfunkel": "Male Duo",
    "The Standells": "Male Group",
    "Count Five": "Male Group",
    "The T-Bones": "Band (Mixed)",
    "Ray Conniff Singers": "Choir (Mixed)",
    "The Shadows of Knight": "Male Group",
    "The Sandpipers": "Mixed Group",
    "James & Bobby Purify": "Male Duo",
    "The Shades of Blue": "Mixed Group",
    "The Vogues": "Male Group",
    "Los Bravos": "Male Group",
    "Mitch Ryder & The Detroit Wheels": "Male Group",
    "The Box Tops": "Male Group",
    "The Music Explosion": "Male Group",
    "Soul Survivors": "Male Group",
    "Sam & Dave": "Male Duo",
    "Every Mother's Son": "Male Group",
    "Buffalo Springfield": "Male Group",
    "The Esquires": "Male Group",
    "Procol Harum": "Male Group",
    "The Casinos": "Male Group",
    "The 5th Dimension": "Mixed Group",
    "Five Americans": "Male Group",
    "Blues Magoos": "Male Group",
    "Bar-Kays": "Male Group",
    "The Royal Guardsmen": "Male Group",
    "The Electric Prunes": "Male Group",
    "The Who": "Male Group",
    "O. C. Smith": "Male",
    "The Delfonics": "Male Group",
    "John Fred & His Playboy Band": "Male Group",
    "Classics IV": "Male Group",
    "Steppenwolf": "Male Group",
    "The Intruders": "Male Group",
    "1910 Fruitgum Company": "Male Group",
    "Sérgio Mendes & Brasil '66": "Mixed Group",
    'Ohio Express': 'Group (Male)',
    'The Crazy World of Arthur Brown': 'Male',
    'Gene & Debbe': 'Mixed',
    'The O\'Kaysions': 'Group (Male)',
    'The Lemon Pipers': 'Group (Male)',
    'Friend & Lover': 'Mixed',
    'Status Quo': 'Group (Male)',
    'Blue Cheer': 'Group (Male)',
    'The Bee Gees': 'Group (Male)',
    'The Fireballs': 'Group (Male)',
    'The Dells': 'Group (Male)',
    'The Human Beinz': 'Group (Male)',
    'Tommy Boyce & Bobby Hart': 'Male',
    'Vanilla Fudge': 'Group (Male)',
    'The Foundations': 'Group (Mixed)',
    'The Sweet Inspirations': 'Group (Female)',
    'The Mills Brothers': 'Group (Male)',
    'Big Brother and the Holding Company': 'Group (Mixed)',
    'The Archies': 'Group (Mixed)',
    'Jr. Walker & The All Stars': 'Group (Male)',
    'Zager & Evans': 'Male',
    'Blood, Sweat & Tears': 'Group (Male)',
    'Smith': 'Group (Mixed)',
    'Mercy': 'Group (Mixed)',
    'The Guess Who': 'Group (Male)',
    'Spiral Starecase': 'Group (Male)',
    '"Mama" Cass Elliot': 'Female',
    'The Winstons': 'Group (Male)',
    'Checkmates, Ltd.': 'Group (Male)',
    'New Colony Six': 'Group (Male)',
    'Motherlode': 'Group (Male)',
    'The Brooklyn Bridge': 'Group (Male)',
    'The Flying Machine': 'Group (Male)',
    'The Cuff Links': 'Group (Male)',
    'Bob Seger System': 'Group (Male)',
    'The Supremes & The Temptations': 'Group (Mixed)',
    'Crazy Elephant': 'Group (Male)',
    'The Originals': 'Group (Male)',
    'Edwin Hawkins Singers': 'Group (Mixed)',
    'The Carpenters': 'Mixed',
    'The Jackson 5': 'Group (Male)',
    'Rare Earth': 'Group (Male)',
    'Vanity Fare': 'Group (Male)',
    'Dawn': 'Group (Mixed)',
    'Sly & the Family Stone': 'Group (Mixed)',
    'Eric Burdon & War': 'Group (Male)',
    'The Moments': 'Group (Male)',
    'Free': 'Group (Male)',
    'Sugarloaf': 'Group (Male)',
    'Shocking Blue': 'Group (Mixed)',
     'The Chakachas': 'Group (Mixed)',
    'T. Rex': 'Male',
    'Dr. Hook & the Medicine Show': 'Male Group',
    'Apollo 100': 'Male Group',
    'The Jimmy Castor Bunch': 'Male Group',
    'Redbone': 'Male Group',
    'Beverly Bremers': 'Female',
    'Original Cast of Godspell': 'Group (Mixed)',
    'The Detroit Emeralds': 'Male Group',
    'The Hillside Singers': 'Group (Mixed)',
    'Stories': 'Male Group',
    'Skylark': 'Group (Mixed)',
    'King Harvest': 'Male Group',
    'Bloodstone': 'Male Group',
    'Seals and Crofts': 'Male',
    'The Doobie Brothers': 'Male Group',
    'New York City': 'Male Group',
    'Loggins and Messina': 'Male',
    'Hurricane Smith': 'Male',
    'Focus': 'Male Group',
    'Tower of Power': 'Male Group',
    'Blue Ridge Rangers': 'Male Group',
    'Deodato': 'Male',
    'Love Unlimited Orchestra': 'Group (Mixed)',
    'MFSB': 'Group (Mixed)',
    'Kool & the Gang': 'Male Group',
    'Blue Magic': 'Male Group',
    'Bo Donaldson and The Heywoods': 'Male Group',
    'Steve Miller Band': 'Male Group',
    'The Hues Corporation': 'Group (Mixed)',
    'Paper Lace': 'Male Group',
    'Mocedades': 'Group (Mixed)',
    'Golden Earring': 'Male Group',
    'The First Class': 'Male Group',
    'Average White Band': 'Male Group',
    'Van McCoy & the Soul City Symphony': 'Group (Mixed)',
    'Ozark Mountain Daredevils': 'Male Group',
    'Pilot': 'Male Group',
    '10cc': 'Male Group',
    'Bazuka': 'Male Group',
    'Styx': 'Male Group',
    'Grand Funk': 'Male Group',
    'KC and the Sunshine Band': 'Male Group',
    'Orleans': 'Male Group',
    'Donny & Marie Osmond': 'Mixed (Male and Female)',
    'Tavares': 'Male Group',
    'Dwight Twilley Band': 'Male Group',
    'Amazing Rhythm Aces': 'Male Group',
    'Bachman–Turner Overdrive': 'Male Group',
    'Disco-Tex and the Sex-O-Lettes': 'Mixed',
    'The Manhattans': 'Male',
    'Hall & Oates': 'Male',
    'Silver Convention': 'Mixed',
    'Andrea True Connection': 'Female',
    'The Sylvers': 'Mixed',
    'Hot Chocolate': 'Male',
    'Nazareth': 'Male',
    'Commodores': 'Male',
    'Rhythm Heritage': 'Mixed',
    'Starbuck': 'Male',
    'Dr. Hook': 'Male',
    'The Brothers Johnson': 'Male',
    'Silver': 'Male',
    'Jefferson Starship': 'Mixed',
    'Parliament': 'Male',
    'Rick Dees & His Cast of Idiots': 'Male',
    'The Emotions': 'Female',
    'Hot': 'Female',
    'Pablo Cruise': 'Male',
    'Foreigner': 'Male',
    "Manfred Mann's Earth Band": 'Male',
    'The Jacksons': 'Male',
    'Brick': 'Male',
    'Bob Seger & The Silver Bullet Band': 'Male',
    'The Marshall Tucker Band': 'Male',
    'Sanford-Townsend Band': 'Male',
    'The Floaters': 'Male',
    'Lord David Dundas': 'Male',
    "Dr. Buzzard's Original Savannah Band": 'Mixed',
    'Heatwave': 'Male',
    'London Symphony Orchestra': 'Mixed',
    'C.J. & Company': 'Mixed',
    'Exile': 'Male',
    'Chic': 'Mixed',
    'Toby Beau': 'Male',
    'The Trammps': 'Male',
    'Lynyrd Skynyrd': 'Male',
    'Patti Smith Group': 'Mixed',
    'Santa Esmeralda': 'Mixed',
    'Boston': 'Male',
    'Eruption': 'Mixed',
    'The Knack': 'Male',
    'Blondie': 'Mixed',
    'Toto': 'Male',
    'The Charlie Daniels Band': 'Male',
    'Earth, Wind & Fire & The Emotions': 'Mixed',
    'McFadden & Whitehead': 'Male',
    'Instant Funk': 'Male',
    'Firefall': 'Male',
    'Lipps Inc': 'Mixed',
    'Air Supply': 'Male',
    'The S.O.S. Band': 'Mixed',
    'The Pretenders': 'Mixed',
    'Ray, Goodman & Brown': 'Male',
    'Pure Prairie League': 'Male',
    'The Dirt Band': 'Male',
    'Charlie Daniels Band': 'Male',
    'Stars on 45': 'Mixed',
    'Champaign': 'Mixed',
    'The Greg Kihn Band': 'Male',
    'Franke and the Knockouts': 'Male',
    'Tierra': 'Male',
    'John Cougar': 'Male',
    'Alabama': 'Male',
    'Devo': 'Male',
    'Survivor': 'Male',
    'The Human League': 'Mixed',
    'Quarterflash': 'Mixed',
    'Tommy Tutone': 'Male',
    'Dazz Band': 'Male',
    "The Go-Go's": 'Female',
    'The Motels': 'Mixed',
    'Huey Lewis and the News': 'Male',
    '.38 Special': 'Male',
    'Royal Philharmonic Orchestra': 'Mixed',
    'Michael Murphey': 'Male',
    'Dexys Midnight Runners': 'Male',
    'Men Without Hats': 'Male',
    'Stray Cats': 'Male',
    'The Clash': 'Male',
    'The Tubes': 'Male',
    'DeBarge': 'Mixed',
    'Spandau Ballet': 'Male',
    'The Romantics': 'Male',
    'Daryl Hall & John Oates': 'Male',
    'Thompson Twins': 'Mixed',
    'Night Ranger': 'Male',
    'Wang Chung': 'Male',
    'Ollie & Jerry': 'Male',
    'John Cougar Mellencamp': 'Male',
    'Ratt': 'Male',
    'Wham!': 'Male',
    'Simple Minds': 'Male',
    'USA for Africa': 'Mixed',
    'New Edition': 'Male',
    'The Honeydrippers': 'Mixed',
    'Animotion': 'Mixed',
    'Sade': 'Female',
    "'Til Tuesday": 'Mixed',
    'Katrina and the Waves': 'Mixed',
    'The Power Station': 'Male',
    'The Time': 'Male',
    'Klymaxx': 'Female',
    'Simply Red': 'Male',
    'Berlin': 'Mixed',
    'Level 42': 'Male',
    'Miami Sound Machine': 'Mixed',
    'Sly Fox': 'Male',
    'The Bangles': 'Female',
    'The Jets': 'Mixed',
    'Orchestral Manoeuvres in the Dark': 'Male',
    'INXS': 'Male',
    'Scritti Politti': 'Male',
    'Cameo': 'Male',
    'Timex Social Club': 'Male',
    'Dream Academy': 'Mixed',
    'Run–D.M.C. feat. Aerosmith': 'Male',
    'Moody Blues': 'Male',
    'Force MDs': 'Male',
    'Whitesnake': 'Male',
    'Crowded House': 'Male',
    'Cutting Crew': 'Male',
    'Georgia Satellites': 'Male',
    'Exposé': 'Female',
    'LeVert': 'Male',
    'The Whispers': 'Male',
    'Breakfast Club': 'Mixed',
    'Pseudo Echo': 'Male',
    "Guns N' Roses": 'Male',
    'Breathe': 'Male',
    'The Escape Club': 'Male',
    'Johnny Hates Jazz': 'Male',
    'Information Society': 'Male',
    'Pretty Poison': 'Mixed',
    'Milli Vanilli': 'Male',
    'Will to Power': 'Mixed',
    'Warrant': 'Male',
    'Sheriff': 'Male',
    'Great White': 'Male',
    'Skid Row': 'Male',
    'The Jeff Healey Band': 'Male',
    'Surface': 'Male',
    'When in Rome': 'Male',
    'Edie Brickell & New Bohemians': 'Mixed',
    'Boys Club': 'Male',
    'Soul II Soul': 'Mixed',
    'Bell Biv DeVoe': 'Male',
    'En Vogue': 'Female',
    'Technotronic': 'Mixed',
    'Katrina and the Waves': 'Mixed',
    'The Power Station': 'Male',
    'The Time': 'Male',
    'Klymaxx': 'Female',
    'Simply Red': 'Male',
    'Berlin': 'Mixed',
    'Level 42': 'Male',
    'Miami Sound Machine': 'Mixed',
    'Sly Fox': 'Male',
    'The Bangles': 'Female',
    'The Jets': 'Mixed',
    'Orchestral Manoeuvres in the Dark': 'Male',
    'INXS': 'Male',
    'Scritti Politti': 'Male',
    'Cameo': 'Male',
    'Timex Social Club': 'Male',
    'Dream Academy': 'Mixed',
    'Run–D.M.C. feat. Aerosmith': 'Male',
    'Moody Blues': 'Male',
    'Force MDs': 'Male',
    'Whitesnake': 'Male',
    'Crowded House': 'Male',
    'Cutting Crew': 'Male',
    'Georgia Satellites': 'Male',
    'Exposé': 'Female',
    'LeVert': 'Male',
    'The Whispers': 'Male',
    'Breakfast Club': 'Mixed',
    'Pseudo Echo': 'Male',
    "Guns N' Roses": 'Male',
    'Breathe': 'Male',
    'The Escape Club': 'Male',
    'Johnny Hates Jazz': 'Male',
    'Information Society': 'Male',
    'Pretty Poison': 'Mixed',
    'Milli Vanilli': 'Male',
    'Will to Power': 'Mixed',
    'Warrant': 'Male',
    'Sheriff': 'Male',
    'Great White': 'Male',
    'Skid Row': 'Male',
    'The Jeff Healey Band': 'Male',
    'Surface': 'Male',
    'When in Rome': 'Male',
    'Edie Brickell & New Bohemians': 'Mixed',
    'Boys Club': 'Male',
    'Soul II Soul': 'Mixed',
    'Bell Biv DeVoe': 'Male',
    'En Vogue': 'Female',
    'Technotronic': 'Mixed',
    'Calloway': 'Male',
    'Snap!': 'Mixed',
    'Seduction': 'Mixed',
    'Linear': 'Male',
    'After 7': 'Male',
    'Digital Underground': 'Male',
    'Tesla': 'Male',
    'Faith No More': 'Male',
    'Black Box': 'Mixed',
    'The Cover Girls': 'Female',
    "A'Me Lorain": 'Female',
    'Jive Bunny and the Mastermixers': 'Mixed',
    'C+C Music Factory': 'Mixed',
    'Extreme': 'Male',
    'Hi-Five': 'Male',
    'Boyz II Men': 'Male',
    'Jesus Jones': 'Male',
    'Natural Selection': 'Male',
    'DJ Jazzy Jeff & The Fresh Prince': 'Male',
    'Rythm Syndicate': 'Male',
    'Another Bad Creation': 'Male',
    'The KLF': 'Mixed',
    'Enigma': 'Mixed',
    'Heavy D & the Boyz': 'Male',
    'Queensrÿche': 'Male',
    'Deee-Lite': 'Mixed',
    'Winger': 'Male',
    'Salt-n-Pepa': 'Female',
    'TLC': 'Female',
    'Red Hot Chili Peppers': 'Male',
    'Joe Public': 'Male',
    'Nirvana': 'Male',
    'Arrested Development': 'Male',
    'P.M. Dawn': 'Male',
    'Shakespears Sister': 'Female',
    'Hammer': 'Male',
    'KWS': 'Mixed',
    'Mint Condition': 'Male',
    'Toad the Wet Sprocket': 'Male',
    'The Heights': 'Mixed',
    'Ugly Kid Joe': 'Male',
    'N2Deep': 'Male',
    'Jade': 'Female',
    'Charles & Eddie': 'Male',
    'Das EFX': 'Male',
    'Tag Team': 'Male',
    'Silk': 'Male',
    'Shai': 'Male',
    'H-Town': 'Male',
    'Duice': 'Male',
    'The Proclaimers': 'Male',
    'Spin Doctors': 'Male',
    'Tony! Toni! Toné!': 'Male',
    '95 South': 'Male',
    '4 Non Blondes': 'Mixed',  # Band with prominent female lead
    'Xscape': 'Female',
    'Portrait': 'Male',
    'Zhané': 'Female',
    'Inner Circle': 'Male',
    'Boy Krazy': 'Female',
    'Restless Heart': 'Male',
    'Captain Hollywood Project': 'Mixed',  # Electronic music project with male and female members
    'Digable Planets': 'Mixed',  # Hip-hop trio with male and female members
    'Green Jellÿ': 'Male',
    'Gin Blossoms': 'Male',
    'Salt-n-Pepa feat. En Vogue': 'Female',  # Collaboration between two female groups
    'Big Mountain': 'Male',
    'Collective Soul': 'Male',
    '10,000 Maniacs': 'Mixed',  # Band with prominent female lead
    'Us3': 'Male',  # Jazz-rap group primarily male
    'Changing Faces': 'Female',
    'Snoop Doggy Dogg': 'Male',
    'Immature': 'Male',
    'Real McCoy': 'Mixed',  # Eurodance group with male and female members
    '69 Boyz': 'Male',
    'Culture Beat': 'Mixed',  # Eurodance group with male and female members
    'Eternal': 'Female',
    'General Public': 'Male',
    'Blues Traveler': 'Male',
    'Soul for Real': 'Male',
    'MoKenStef': 'Female',
    '4 P.M.': 'Male',
    'Corona': 'Female',  # Although Corona is often seen as a solo female artist, it's a project fronted by a female vocalist
    'Del Amitri': 'Male',
    'Subway feat. 702': 'Mixed',  # Collaboration between a male group (Subway) and a female group (702)
    'Better Than Ezra': 'Male',
    'Rednex': 'Mixed',  # Music group with male and female members
    'N II U': 'Male',
    'The Tony Rich Project': 'Male',
    "Quad City DJ's": 'Mixed',  # Group with male and female members
    'La Bouche': 'Mixed',  # Eurodance duo with male and female members
    'Goo Goo Dolls': 'Male',
    'No Mercy': 'Male',
    'Dishwalla': 'Male',
    'Deep Blue Something': 'Male',
    '3T': 'Male',
    'The Smashing Pumpkins': 'Mixed',  # Rock band with male and female members
    'Oasis': 'Male',
    'No Doubt': 'Mixed',  # Band with prominent female lead and male members
    'Planet Soul': 'Mixed',
    "Ghost Town DJ's": 'Male',
    'Third Eye Blind': 'Male',
    'Dru Hill': 'Male',
    'Sister Hazel': 'Male',
    '112': 'Male',
    'Allure feat. 112': 'Mixed',  # Allure (Female) feat. 112 (Male)
    '98 Degrees': 'Male',
    '702': 'Female',
    'White Town': 'Male',  # Solo project by Jyoti Mishra
    'Chumbawamba': 'Mixed',
    'Various Artists': 'Mixed',
    'Luscious Jackson': 'Female',
    'SWV feat. Puff Daddy': 'Mixed',  # SWV (Female) feat. Puff Daddy (Male)
    'Next': 'Male',
    'Mr. President': 'Mixed',
    'The Blackout All-Stars': 'Male',
    'Public Announcement': 'Male',
    'NSYNC': 'Male',
    'Five': 'Male',
    'Voices of Theory': 'Male',
    'K. P. & Envyi': 'Female',
    'Luke feat. No Good But So Good': 'Mixed',  # Luke (Male) feat. No Good But So Good (Female)
    'Nu Flavor': 'Male',
    'Sixpence None the Richer': 'Mixed',  # Lead by Leigh Nash (Female)
    'Sugar Ray': 'Male',
    'Smash Mouth': 'Male',
    'Matchbox 20': 'Male',
    'Blaque': 'Female',
    'Len': 'Mixed',
    'Mo Thugs': 'Male',
    'Fastball': 'Male',
    'Naughty by Nature feat. Zhané': 'Mixed',  # Naughty by Nature (Male) feat. Zhané (Female)
    'Citizen King': 'Male',
    'Alabama and NSYNC': 'Mixed',  # Alabama (Male) and NSYNC (Male)
    'Santana feat. The Product G&B': 'Mixed',  # Santana (Mixed) feat. The Product G&B (Male)
    'Matchbox Twenty': 'Male',  # Duplicate of Matchbox 20
    '3 Doors Down': 'Male',
    'Pink': 'Female',
    'Ruff Endz': 'Male',
    'Blink 182': 'Male',
    'Eiffel 65': 'Male',
    'Everclear': 'Male',
    'Filter': 'Male',
    'Son by Four': 'Male',
    'soulDecision feat. Thrust': 'Mixed',  # soulDecision (Male) feat. Thrust (Male)
    "Lil' Bow Wow feat. Xscape": 'Mixed',  # Lil' Bow Wow (Male) feat. Xscape (Female)
    'Dixie Chicks': 'Female',
    'Mary Mary': 'Female',
    'Staind': 'Male',
    'Dream': 'Female',
    'O-Town': 'Male',
    'Evan and Jaron': 'Male',
    '3LW': 'Female',
    'Brooks & Dunn': 'Male',
    "Lil' Romeo": 'Male',
    'Willa Ford': 'Female',
    'Diamond Rio': 'Mixed',
    'Eve 6': 'Male',
    'Fuel': 'Male',
    'Linkin Park': 'Male',
    'Puddle of Mudd': 'Male',
    'Jimmy Eat World': 'Male',
    'Default': 'Male',
    'Dirty Vegas': 'Male',
    'B2K and P. Diddy': 'Mixed',
    'Black Eyed Peas': 'Mixed',
    'Young Gunz': 'Male',
    'Maroon 5': 'Male',
    'Los Lonely Boys': 'Male',
    'D12': 'Male',
    'Santana feat. Alex Band or Chad Kroeger': 'Mixed',
    'Jet': 'Male',
    'The Black Eyed Peas': 'Mixed',
    'Weezer': 'Male',
    'DHT feat. Edmee': 'Mixed',
    'Ying Yang Twins': 'Male',
    'Foo Fighters': 'Male',
    'The Fray': 'Male',
    'Panic! at the Disco': 'Male',
    'Dem Franchize Boyz feat. Peanut and Charlay': 'Male',
    'Rascal Flatts': 'Mixed',
    'Snow Patrol': 'Male',
    'Pussycat Dolls': 'Female',
    "Plain White T's": 'Male',
    'Shop Boyz': 'Male',
    'Mims': 'Male',
    "Soulja Boy Tell 'Em": 'Male',
    'The Red Jumpsuit Apparatus': 'Male',
    'My Chemical Romance': 'Male',
    'Hellogoodbye': 'Male',
    'Metro Station': 'Male',
    'Secondhand Serenade': 'Male',
    'Jonas Brothers': 'Male',
    'Shinedown': 'Male',
    '3OH!3': 'Male',
    'Zac Brown Band': 'Mixed',
    'Lady Antebellum': 'Mixed',
    'Selena Gomez & the Scene': 'Mixed',
    'Cali Swag District': 'Male',
    'Foster the People': 'Male',
    'Eli Young Band': 'Male',
    'Thompson Square': 'Mixed',
    'Fun feat. Janelle Monáe': 'Mixed',
    'The Wanted': 'Male',
    'One Direction': 'Male',
    'Fun': 'Male',
    'Karmin': 'Mixed',
    'Little Big Town': 'Mixed',
    'The Lumineers': 'Mixed',
    'Imagine Dragons': 'Male',
    'Cash Out': 'Male',
    'Awolnation': 'Male',
    'Capital Cities': 'Male',
    'Of Monsters and Men': 'Mixed',
    'Ylvis': 'Male',
    'Florida Georgia Line': 'Male',
    'Magic!': 'Male',
    'Bastille': 'Male',
    'Nico & Vinz': 'Male',
    'Disclosure feat. Sam Smith': 'Mixed',
    'American Authors': 'Male',
    'MKTO': 'Male',
    'Echosmith': 'Mixed',
    'Rixton': 'Male',
    'The Neighbourhood': 'Male',
    '5 Seconds of Summer': 'Male',
    'X Ambassadors': 'Male',
    'Rae Sremmurd': 'Male',
    'Lukas Graham': 'Male',
    'Fifth Harmony feat. Ty Dolla Sign': 'Mixed',
    'DNCE': 'Mixed',
    'The Chainsmokers and Coldplay': 'Mixed',
    'Migos feat. Lil Uzi Vert': 'Male',
    'The Chainsmokers': 'Male',
    'Portugal. The Man': 'Male',
    'Migos': 'Male',
    'Ayo & Teo': 'Male',
    'Marian Hill': 'Mixed',
    'Cheat Codes feat. Demi Lovato': 'Mixed',
    'Dan + Shay': 'Male',
    'Max feat. Gnash': 'Male',
    'City Girls': 'Female',
    'Shaed': 'Mixed',
    'Pinkfong': 'Not Applicable',  # Being a company/brand, it doesn't fit traditional gender categories.
    'BTS': 'Male',
    'Old Dominion': 'Male',
    'Surfaces': 'Male',
    'Maddie & Tae': 'Female',
    'Glass Animals': 'Male',
    'CJ': 'Male',
    'AJR': 'Male',
    'Måneskin': 'Mixed',
    'Mooski': 'Male',
    'Ritt Momney': 'Male',
    'Parmalee': 'Male',
    'Fuerza Regida and Grupo Frontera': 'Male',
    'Oliver Anthony Music': 'Male',
    'Fifty Fifty': 'Female',
     'Kenny Rogers & The First Edition': 'Mixed',
    'Chairmen of the Board': 'Male Group',
    'Marmalade': 'Male Group',
    'The Jaggerz': 'Male Group',
    "Alive N Kickin'": 'Mixed Group',
    'Badfinger': 'Male Group',
    'The Four Tops': 'Male Group',
    'Frijid Pink': 'Male Group',
    'White Plains': 'Male Group',
    'The Brotherhood of Man': 'Mixed Group',
    'Santana': 'Male Group',
    'The Spinners': 'Male Group',
    'Mountain': 'Male Group',
    'Tee-Set': 'Male Group',
    '100 Proof (Aged in Soul)': 'Male Group',
    'The Pipkins': 'Male Duo',
    'Pacific Gas & Electric': 'Male Group',
    'The New Seekers': 'Mixed Group',
    'The Osmonds': 'Male Group',
    'Raiders': 'Male Group',
    'Honey Cone': 'Female Group',
    'Cornelius Brothers & Sister Rose': 'Mixed Group',
    'The Free Movement': 'Mixed Group',
    'Ocean': 'Mixed Group',
    'Daddy Dewdrop': 'Male',
    'Hamilton, Joe Frank & Reynolds': 'Male Group',
    'Nitty Gritty Dirt Band': 'Male Group',
    'The Fuzz': 'Mixed Group',
    'The Dramatics': 'Male Group',
    'The Partridge Family': 'Mixed Group',
    'The Bells': 'Mixed Group',
    'The Stampeders': 'Male Group',
    'Brewer & Shipley': 'Male Duo',
    '8th Day': 'Male Group',
    'The Beginning of the End': 'Male Group',
    'Matthews Southern Comfort': 'Male Group',
    'The Buoys': 'Male Group',
    'Paul Stookey': 'Male',
    'Wadsworth Mansion': 'Male Group',
    'Brenda & the Tabulations': 'Mixed Group',
    'Looking Glass': 'Male Group',
    'Gallery': 'Male Group',
    'The Stylistics': 'Male Group',
    'Mouth & MacNeal': 'Mixed Duo',
    'America': 'Male Group',
    'Hot Butter': 'Male',
    'The Main Ingredient': 'Male Group',
    'Climax': 'Mixed Group',
    'Raspberries': 'Male Group',
    'Dr. Hook & The Medicine Show': 'Male Group',
    "The O'Jays": 'Male'
    
    
}

# Return the dictionary to confirm the entries
# artist_gender_annotations

In [75]:
unknown_gender['Artist Gender'] = unknown_gender['Artist'].map(artist_gender_annotations)

In [76]:
unknown_gender

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,16,Alley Oop,The Hollywood Argyles,Group,Male Group,NaN,Oopoop oop oopoop\n\n\n\nThere's a man in the ...,en
1,1960,26,Save the Last Dance for Me,The Drifters,Group,Male Group,['Soul'],You can dance\nEvery dance with the guy\nWho g...,en
2,1960,36,The Big Hurt,Toni Fisher,Unknown,Female,NaN,Now it begins now that you're gone\nNeedles an...,en
3,1960,44,Mule Skinner Blues,The Fendermen,Group,Male Duo,NaN,Well good morning Captain\nWell good morning t...,en
4,1960,53,Theme from The Apartment,Ferrante & Teicher,Collab,Male Duo,NaN,1 ContributorTheme From The Apartment LyricsMa...,en
...,...,...,...,...,...,...,...,...,...
1347,2023,44,Cupid,Fifty Fifty,Group,Female,"['Girl Group', 'K-Pop']",Intro Aran\nLa la la lalala\nLa lalala la lala...,en
1348,2023,73,Bebe Dame,Fuerza Regida and Grupo Frontera,Collab,Male,NaN,Intro Jesús Paz Payo Solís\nFierro\nÁnimo\nDe...,es
1349,2023,78,Rich Men North of Richmond,Oliver Anthony Music,Unknown,Male,NaN,I've been sellin' my soul workin' all day\nOve...,en
1350,2023,84,Last Christmas,Wham!,Group,Male,"['Pop', 'Dance-Pop']",Happy Christmas\nAh ahah\nOohwoah\nOhoh\n\n\nL...,en


In [77]:
unknown_gender['Artist Gender'].value_counts()

Artist Gender
Male                       582
Male Group                 296
Mixed                      189
Female                     102
Group (Male)                47
Mixed Group                 43
Female Group                34
Group (Mixed)               22
Male Duo                    21
Mixed Duo                    5
Band (Mixed)                 4
Choir (Mixed)                2
Mixed (Male and Female)      2
Orchestra (Mixed)            1
Group (Female)               1
Not Applicable               1
Name: count, dtype: int64

In [78]:
unknown_gender.isna().sum()

Year                   0
Position               0
Song Title             0
Artist                 0
Artist Type            0
Artist Gender          0
Genres               653
lyrics               113
detected_language    113
dtype: int64

In [79]:
def unify_gender_classes(gender):
    # Convert NaN (float) to a string for safe processing
    gender = str(gender)
    
    # Now we can safely check for substrings
    if 'Male' in gender or 'Group (Male)' in gender or 'Male Group' in gender or 'Male Duo' in gender:
        return 'Male'
    elif 'Female' in gender or 'Group (Female)' in gender or 'Female Group' in gender:
        return 'Female'
    elif ('Mix' in gender or 'Mixed' in gender or 'Group (Mixed)' in gender 
    or 'Mixed Duo' in gender or 'Band (Mixed)' in gender or 'Choir (Mixed)' in gender 
    or 'Mixed (Male and Female)' in gender or 'Mixed Group' in gender 
    or 'Orchestra (Mixed)' in gender):
        return 'Mix'
    else:
        return gender  # Retain original for 'Not Applicable' and 'Needs Clarification'

# Apply the function to the 'Artist Gender' column
unknown_gender['Artist Gender'] = unknown_gender['Artist Gender'].apply(unify_gender_classes)

In [80]:
unknown_gender[unknown_gender['Artist Gender']=='Not Applicable']

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
1323,2019,75,Baby Shark,Pinkfong,Group,Not Applicable,NaN,Baby shark doodoodoodoodoodoo\nBaby shark dood...,so


In [81]:
unknown_gender

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,16,Alley Oop,The Hollywood Argyles,Group,Male,NaN,Oopoop oop oopoop\n\n\n\nThere's a man in the ...,en
1,1960,26,Save the Last Dance for Me,The Drifters,Group,Male,['Soul'],You can dance\nEvery dance with the guy\nWho g...,en
2,1960,36,The Big Hurt,Toni Fisher,Unknown,Female,NaN,Now it begins now that you're gone\nNeedles an...,en
3,1960,44,Mule Skinner Blues,The Fendermen,Group,Male,NaN,Well good morning Captain\nWell good morning t...,en
4,1960,53,Theme from The Apartment,Ferrante & Teicher,Collab,Male,NaN,1 ContributorTheme From The Apartment LyricsMa...,en
...,...,...,...,...,...,...,...,...,...
1347,2023,44,Cupid,Fifty Fifty,Group,Female,"['Girl Group', 'K-Pop']",Intro Aran\nLa la la lalala\nLa lalala la lala...,en
1348,2023,73,Bebe Dame,Fuerza Regida and Grupo Frontera,Collab,Male,NaN,Intro Jesús Paz Payo Solís\nFierro\nÁnimo\nDe...,es
1349,2023,78,Rich Men North of Richmond,Oliver Anthony Music,Unknown,Male,NaN,I've been sellin' my soul workin' all day\nOve...,en
1350,2023,84,Last Christmas,Wham!,Group,Male,"['Pop', 'Dance-Pop']",Happy Christmas\nAh ahah\nOohwoah\nOhoh\n\n\nL...,en


In [82]:
unknown_gender['Artist Gender'].value_counts()

Artist Gender
Male              948
Mix               266
Female            137
Not Applicable      1
Name: count, dtype: int64

In [43]:
unknown_gender.shape[0]

1352

In [84]:
unknown_gender[unknown_gender['Artist Type']=='Unknown']

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
2,1960,36,The Big Hurt,Toni Fisher,Unknown,Female,NaN,Now it begins now that you're gone\nNeedles an...,en
7,1960,61,Forever,The Little Dippers,Unknown,Mix,NaN,Hold me kiss me\nWhisper sweetly\nThat you lov...,en
52,1963,7,Fingertips,Little Stevie Wonder,Unknown,Male,NaN,NaN,NaN
54,1963,10,Sukiyaki,Kyu Sakamoto,Unknown,Male,NaN,Verse 1\n上を向いて歩こう\n涙がこぼれないように\n思い出す春の日\n一人ぼっちの...,ja
60,1963,20,I Will Follow Him,Little Peggy March,Unknown,Female,NaN,I love him I love him I love him\nAnd where He...,en
...,...,...,...,...,...,...,...,...,...
1242,2012,92,Cashin' Out,Cash Out,Unknown,Male,NaN,NaN,NaN
1260,2013,93,Downtown,Lady Antebellum,Unknown,Mix,NaN,Well all the parties on the streets are talkin...,en
1277,2014,81,Bartender,Lady Antebellum,Unknown,Mix,NaN,8 oclock on Friday night Im still at home\nAll...,en
1339,2021,50,Whoopty,CJ,Unknown,Male,NaN,Heavy on the SSO shit man\nLoyalty over royalt...,en


In [85]:
unknown_gender['Artist Gender'].value_counts()

Artist Gender
Male              948
Mix               266
Female            137
Not Applicable      1
Name: count, dtype: int64

In [86]:
unknown_gender['Artist Type'].value_counts()

Artist Type
Group        1143
Unknown       131
Collab         67
Person          8
Orchestra       2
Other           1
Name: count, dtype: int64

In [87]:
unknown_gender.isna().sum()

Year                   0
Position               0
Song Title             0
Artist                 0
Artist Type            0
Artist Gender          0
Genres               653
lyrics               113
detected_language    113
dtype: int64

In [88]:
billboard_artist_lyrics = billboard_artist_lyrics.dropna(subset = ['Artist Gender'])

In [89]:
billboard_artist_lyrics_ = pd.concat([billboard_artist_lyrics, unknown_gender], axis = 0)

In [90]:
billboard_artist_lyrics_ = billboard_artist_lyrics_.sort_values(by = ['Year', 'Position'])
billboard_artist_lyrics_.reset_index(drop = True, inplace = True)

In [91]:
billboard_artist_lyrics_

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...,en
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...,en
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...,en
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...,en
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...,en
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mixed,"['Pop', 'Pop Rock', 'Latin Pop']",NaN,NaN
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",NaN,NaN
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",Go Grizz\nAh \nWhat's happenin'? \nBig Latto \...,en
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",Money\n\n\nLove is just a history that they ma...,en


In [92]:
billboard_artist_lyrics_['Artist Gender'].value_counts()

Artist Gender
Male              4173
Female            1433
Mixed              502
Mix                266
Non-binary          25
Not Applicable       1
Name: count, dtype: int64

In [93]:
# billboard_artist_lyrics_[billboard_artist_lyrics_['Artist Gender']=='Mixed'] = 'Mix'
billboard_artist_lyrics_.loc[billboard_artist_lyrics_['Artist Gender'] == 'Mixed', 'Artist Gender'] = 'Mix'


In [94]:
billboard_artist_lyrics_['Artist Gender'].value_counts(normalize=True)

Artist Gender
Male              0.652031
Female            0.223906
Mix               0.120000
Non-binary        0.003906
Not Applicable    0.000156
Name: proportion, dtype: float64

In [95]:
billboard_artist_lyrics_.isna().sum()

Year                    0
Position                0
Song Title              0
Artist                  0
Artist Type             0
Artist Gender           0
Genres               1232
lyrics                453
detected_language     453
dtype: int64

In [96]:
billboard_artist_lyrics_['Artist Type'].value_counts()

Artist Type
Person       3064
Group        2083
Collab       1119
Unknown       131
Orchestra       2
Other           1
Name: count, dtype: int64

In [97]:
# billboard_artist_lyrics_[billboard_artist_lyrics_['Artist Type'] == 'Unknown']['Artist'].unique().tolist()

In [98]:
unknown_artist_type = billboard_artist_lyrics_[billboard_artist_lyrics_['Artist Type'] == 'Unknown']

In [99]:
unknown_artist_type

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
35,1960,36,The Big Hurt,Toni Fisher,Unknown,Female,NaN,Now it begins now that you're gone\nNeedles an...,en
60,1960,61,Forever,The Little Dippers,Unknown,Mix,NaN,Hold me kiss me\nWhisper sweetly\nThat you lov...,en
306,1963,7,Fingertips,Little Stevie Wonder,Unknown,Male,NaN,NaN,NaN
309,1963,10,Sukiyaki,Kyu Sakamoto,Unknown,Male,NaN,Verse 1\n上を向いて歩こう\n涙がこぼれないように\n思い出す春の日\n一人ぼっちの...,ja
319,1963,20,I Will Follow Him,Little Peggy March,Unknown,Female,NaN,I love him I love him I love him\nAnd where He...,en
...,...,...,...,...,...,...,...,...,...
5291,2012,92,Cashin' Out,Cash Out,Unknown,Male,NaN,NaN,NaN
5392,2013,93,Downtown,Lady Antebellum,Unknown,Mix,NaN,Well all the parties on the streets are talkin...,en
5480,2014,81,Bartender,Lady Antebellum,Unknown,Mix,NaN,8 oclock on Friday night Im still at home\nAll...,en
6149,2021,50,Whoopty,CJ,Unknown,Male,NaN,Heavy on the SSO shit man\nLoyalty over royalt...,en


#### Manual annotation for the Artist Type:

In [100]:
artist_types = {
    'Person': [
        'Toni Fisher',
        'Little Stevie Wonder',
        'Kyu Sakamoto',
        'Little Peggy March',
        "Jumpin' Gene Simmons",
        'SSgt Barry Sadler',
        'O. C. Smith',
        'Paul Stookey',
        'Hurricane Smith',
        'Deodato',
        'John Cougar',
        'Michael Murphey',
        'John Cougar Mellencamp',
        'Snoop Doggy Dogg',
        'CJ',
        'Oliver Anthony Music',
         "A'Me Lorain",
        'The Tony Rich Project',
        "Lil' Romeo",
        "Soulja Boy Tell 'Em",
        'Cash Out'
    ],
    'Group': [
        'The Little Dippers',
        'Ramsey Lewis Trio',
        'The Young Rascals',
        'Ray Conniff Singers',
        'The Shades of Blue',
        'Five Americans',
        'Bar-Kays',
        '1910 Fruitgum Company',
        'The Bee Gees',
        'Smith',
        '"Mama" Cass Elliot',
        'New Colony Six',
        'Bob Seger System',
        'The Carpenters',
        'The Jackson 5',
        'The Four Tops',
        'The Brotherhood of Man',
        'Raiders',
        'The Chakachas',
        'Original Cast of Godspell',
        'The Detroit Emeralds',
        'The First Class',
        'Ozark Mountain Daredevils',
        'Grand Funk',
        'Amazing Rhythm Aces',
        'Andrea True Connection',
        'Sanford-Townsend Band',
        'Lord David Dundas',
        'Charlie Daniels Band',
        'The Greg Kihn Band',
        'Dream Academy',
        'Moody Blues',
        'Force MDs',
        'Georgia Satellites',
        'Goo Goo Dolls',
        '98 Degrees',
        'Matchbox 20',
        'Blink 182',
        'Dixie Chicks',
        'The Black Eyed Peas',
        'Pussycat Dolls',
        'N II U',
        'Lady Antebellum'
    ],
    'Collab': [
        'B. J. Thomas & the Triumphs',
        'Gene & Debbe'
    ],
    'Unknown': [],
    # Assuming 'Mix', 'Orchestra', and 'Other' based on the provided list might not have clear examples.
    'Mix': [],
    'Orchestra': [],
    'Other': []
}

In [101]:
artist_to_type = {}
for type_, artists in artist_types.items():
    for artist in artists:
        artist_to_type[artist] = type_


In [102]:
billboard_artist_lyrics_['Artist Type'] = billboard_artist_lyrics_['Artist'].map(artist_to_type).fillna(billboard_artist_lyrics_['Artist Type'])



In [103]:
billboard_artist_lyrics_

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...,en
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...,en
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...,en
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...,en
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...,en
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mix,"['Pop', 'Pop Rock', 'Latin Pop']",NaN,NaN
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",NaN,NaN
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",Go Grizz\nAh \nWhat's happenin'? \nBig Latto \...,en
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",Money\n\n\nLove is just a history that they ma...,en


In [104]:
billboard_artist_lyrics_.shape

(6400, 9)

In [105]:
billboard_artist_lyrics_['Artist Type'].value_counts()

Artist Type
Person       3094
Group        2182
Collab       1121
Orchestra       2
Other           1
Name: count, dtype: int64

In [106]:
billboard_artist_lyrics_

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
0,1960,1,Theme from A Summer Place,Percy Faith,Person,Male,NaN,There's a summer place\nWhere it may rain or s...,en
1,1960,2,He'll Have to Go,Jim Reeves,Person,Male,['Country'],Put your sweet lips a little closer to the pho...,en
2,1960,3,Cathy's Clown,The Everly Brothers,Group,Male,"['Rockabilly', 'Rock & Roll']",Don't want your love anymore\nDon't want your ...,en
3,1960,4,Running Bear,Johnny Preston,Person,Male,['Traditional pop music'],On the bank of the river\nStood Running Bear y...,en
4,1960,5,Teen Angel,Mark Dinning,Person,Male,NaN,Teen Angel\nTeen Angel\nTeen Angel\n\n\nThat f...,en
...,...,...,...,...,...,...,...,...,...
6395,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira,Collab,Mix,"['Pop', 'Pop Rock', 'Latin Pop']",NaN,NaN
6396,2023,97,Meltdown,Travis Scott feat. Drake,Collab,Male,"['English', 'R&B', 'Contemporary R&B', 'Altern...",NaN,NaN
6397,2023,98,Put It on da Floor Again,Latto feat. Cardi B,Collab,Female,"['Alternative Pop', 'Awful', 'Hip Hop']",Go Grizz\nAh \nWhat's happenin'? \nBig Latto \...,en
6398,2023,99,Bloody Mary,Lady Gaga,Person,Female,"['Dance', 'Synth-Pop', 'Disco', 'Pop', 'Pop Ro...",Money\n\n\nLove is just a history that they ma...,en


In [107]:
billboard_artist_lyrics_[billboard_artist_lyrics_['Artist Type']=='Other']

,Year,Position,Song Title,Artist,Artist Type,Artist Gender,Genres,lyrics,detected_language
3771,1997,72,ESPN Presents The Jock Jam,Various Artists,Other,Mix,"['Special Purpose', 'Compilation', 'Special Pu...",NaN,NaN


In [108]:
billboard_artist_lyrics_.to_csv('final_billboard_songs_1960_2023_with_artist_gender_genre_lyrics.csv', index = None)